## Example: Converting a Segmentation Stack for Use with TrackGardener

This notebook provides a practical example of converting a segmentation stack - where each object’s value corresponds to its unique TrackID - to a TrackGardener database using the configuration file `Fluo-N2DL-HeLa-01_config_trackID.yaml`.

In this workflow, relationships between parent and offspring tracks are inferred based on their proximity in time and space. This approach is generally error-prone and not recommended for most applications. For more accurate lineage relationships, use an explicit graph structure; see labels_and_geff_to_TrackGardener for details of importing the graph structure from the `geff` format.

You can reuse the configuration file demonstrated here with the TrackGardener plugin itself. Just be sure to specify the correct paths to your imaging dataset and the TrackGardener database within the config file. Using absolute (rather than relative as for the purpose of this example) paths is the safest choice to ensure your files are correctly located.

In [1]:
import dask.array as da
import yaml

from track_gardener.converters.track_array_2_gardener import (
    assign_parent_offspring_relationships,
    convert_array_segmentations_to_db,
)

In [ ]:
# pathways to the segmentation stack and configuration file

segm_path = './Fluo-N2DL-HeLa-01_segm.zarr/labels'
config_path = './Fluo-N2DL-HeLa-01_config_trackID.yaml'

In [3]:
# read segmentation array
segm_array = da.from_zarr(segm_path)

In [6]:
# read in configuration file
with open(config_path) as file:
    config = yaml.safe_load(file)

In [7]:
# convert and save to Track Gardener database
convert_array_segmentations_to_db(segm_array, config)

2025-07-25 10:13:48.195 | INFO     | track_gardener.converters.track_array_2_gardener:convert_labeled_frame_to_cells:89 - Found 2 labeled objects at t=0
2025-07-25 10:13:48.267 | INFO     | track_gardener.converters.track_array_2_gardener:convert_labeled_frame_to_cells:89 - Found 2 labeled objects at t=1
2025-07-25 10:13:48.322 | INFO     | track_gardener.converters.track_array_2_gardener:convert_labeled_frame_to_cells:89 - Found 2 labeled objects at t=2
2025-07-25 10:13:48.357 | INFO     | track_gardener.converters.track_array_2_gardener:convert_labeled_frame_to_cells:89 - Found 3 labeled objects at t=3
2025-07-25 10:13:48.425 | INFO     | track_gardener.converters.track_array_2_gardener:convert_labeled_frame_to_cells:89 - Found 3 labeled objects at t=4
2025-07-25 10:13:48.504 | INFO     | track_gardener.converters.track_array_2_gardener:convert_labeled_frame_to_cells:89 - Found 3 labeled objects at t=5
2025-07-25 10:13:48.590 | INFO     | track_gardener.converters.track_array_2_garde

In [8]:
# guess parent-offspring relationships by space-time proximity
assign_parent_offspring_relationships(db_path=config['database']['path'], parent_radius=100)

2025-07-25 10:14:03.618 | INFO     | track_gardener.converters.track_array_2_gardener:assign_parent_offspring_relationships:172 - Connecting to database at Fluo-N2DL-HeLa-01_conversion_result.db
2025-07-25 10:14:03.629 | INFO     | track_gardener.converters.track_array_2_gardener:assign_parent_offspring_relationships:182 - Found 12 candidate offspring tracks
2025-07-25 10:14:03.644 | DEBUG    | track_gardener.converters.track_array_2_gardener:assign_parent_offspring_relationships:235 - Assigned parent 204 to child 207 (dist=10.00)
2025-07-25 10:14:03.655 | DEBUG    | track_gardener.converters.track_array_2_gardener:assign_parent_offspring_relationships:235 - Assigned parent 207 to child 208 (dist=20.40)
2025-07-25 10:14:03.660 | DEBUG    | track_gardener.converters.track_array_2_gardener:assign_parent_offspring_relationships:235 - Assigned parent 220 to child 211 (dist=25.00)
2025-07-25 10:14:03.664 | DEBUG    | track_gardener.converters.track_array_2_gardener:assign_parent_offspring_r